# 데이터 저장소 사용

이전 랩에서는 Azure ML 작업 영역에 연결하고 실험 스크립트가 실행되는 로컬 CSV 파일의 데이터를 기반으로 간단한 실험을 실행했습니다. 이번에는 클라우드에 저장된 데이터를 사용해 보겠습니다.

> **중요**: 이 Notebook의 코드에서는 [랩 4A](labdocs/Lab04A.md)의 처음 두 작업을 완료했다고 가정합니다. 해당 작업을 완료하지 않았다면 지금 완료하세요.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 작업 영역에서 데이터 저장소 보기

작업 영역에는 [이전 작업](labdocs/Lab04A.md)에서 만든 **aml_data** 데이터 저장소를 비롯한 여러 데이터 저장소가 포함되어 있습니다.

다음 코드를 실행하여 *기본* 데이터 저장소를 검색한 다음 모든 데이터 저장소를 나열하고 기본 데이터 저장소를 표시합니다.

In [ ]:
# 기본 데이터 저장소 가져오기
default_ds = ws.get_default_datastore()

# 모든 데이터 저장소를 열거하고 기본 데이터 저장소 표시
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

## 사용할 데이터 저장소 가져오기

**aml_data** 데이터 저장소를 사용할 것이므로 이름으로 이 데이터 저장소를 가져와야 합니다.

In [ ]:
from azureml.core import Datastore

aml_datastore = Datastore.get(ws, 'aml_data')
print(aml_datastore.name,":", aml_datastore.datastore_type + " (" + aml_datastore.account_name + ")")

## 기본 데이터 저장소 설정

이 과정에서는 **aml_data** 데이터 저장소를 주로 사용할 것이므로, 편의상 이 저장소를 기본 데이터 저장소로 설정할 수 있습니다.

In [ ]:
ws.set_default_datastore('aml_data')
default_ds = ws.get_default_datastore()
print(default_ds.name)

## 데이터 저장소에 데이터 업로드

사용할 데이터 저장소를 확인했으므로 이제 로컬 파일 시스템에서 파일을 업로드할 수 있습니다. 그러면 실험 스크립트가 실제로 실행되는 위치에 관계없이 작업 영역에서 실행 중인 실험에서 해당 파일에 액세스할 수 있습니다.

In [ ]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                       target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                       overwrite=True, # 이름이 같은 기존 파일 바꾸기
                       show_progress=True)

## 데이터 저장소에서 모델 학습 진행

위의 코드 셀에서 파일을 업로드하면 코드가 *데이터 참조*를 반환합니다. 데이터 참조를 활용하면 스크립트가 실행되는 위치에 관계없이 데이터 저장소의 폴더 경로를 스크립트에 전달할 수 있습니다. 따라서 스크립트가 데이터 저장소 위치의 데이터에 액세스할 수 있습니다.

다음 코드는 당뇨병 CSV 파일을 업로드한 **diabetes-data** 폴더의 참조를 가져온 다음 *다운로드*용 데이터 참조를 구체적으로 구성합니다. 즉, 이 코드를 사용하면 데이터 참조가 사용되는 컴퓨팅 컨텍스트에 폴더의 내용을 다운로드할 수 있습니다. 로컬 컴퓨팅에서 처리할 소량의 데이터는 다운로드하는 방식이 효율적입니다. 원격 컴퓨팅을 사용할 때는 데이터 참조를 구성하여 데이터 저장소 위치를 *탑재*한 다음 데이터 원본에서 데이터를 직접 읽을 수도 있습니다.

> **추가 정보**: 데이터 저장소 사용에 대한 자세한 내용은 [Azure ML 설명서](https://docs.microsoft.com/azure/machine-learning/how-to-access-data)를 참조하세요.

In [ ]:
data_ref = default_ds.path('diabetes-data').as_download(path_on_compute='diabetes_data')
print(data_ref)

학습 스크립트에서 데이터 참조를 사용하려면 데이터 참조에 해당하는 매개 변수를 정의해야 합니다. 아래의 두 코드 셀을 실행하여 다음 항목을 만듭니다.

1. **diabetes_training_from_datastore** 폴더
2. 분류 모델을 학습시키는 스크립트. 이 스크립트는 전달된 데이터 참조 매개 변수가 참조하는 폴더의 모든 CSV 파일에 포함되어 있는 학습 데이터를 사용합니다.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_from_datastore'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created.')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import os
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 매개 변수 가져오기
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder reference')
args = parser.parse_args()
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 데이터 참조에서 당뇨병 데이터 로드
data_folder = args.data_folder
print("Loading data from", data_folder)
# 모든 파일을 로드하고 해당 내용을 단일 데이터 프레임으로 연결
all_files = os.listdir(data_folder)
diabetes = pd.concat((pd.read_csv(os.path.join(data_folder,csv_file)) for csv_file in all_files))

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

스크립트는 전달된 데이터 참조의 학습 데이터를 매개 변수로 로드합니다. 따라서 실험을 실행할 때 파일 참조를 전달하도록 스크립트 매개 변수를 설정하기만 하면 됩니다.

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# 매개 변수 설정
script_params = {
    '--regularization': 0.1, # 정규화 비율
    '--data-folder': data_ref # 데이터 저장소에서 파일을 다운로드하는 데이터 참조
}


# 추정기 만들기
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local'
                   )

# 실험 만들기
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 실험 실행
run = experiment.submit(config=estimator)
# 실행 중에 실행 세부 정보 표시
RunDetails(run).show()
run.wait_for_completion()

실험을 처음 실행할 때는 Python 환경을 설정하는 데 시간이 다소 걸릴 수 있습니다. 후속 실행은 더 빠르게 진행됩니다.

실험이 완료되면 위젯에서 **azureml-logs/70_driver_log.txt** 출력 로그를 표시하여 데이터 파일이 다운로드되었는지 확인합니다.

모든 실험에서와 마찬가지로 [Azure ML Studio](https://ml.azure.com)에서 실험 실행의 세부 정보를 확인할 수 있으며, 생성된 파일과 메트릭을 검색하는 코드를 작성할 수도 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

이번에도 실험을 통해 학습된 모델을 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (using Datastore)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 등록된 모델 목록 표시
print("Registered Models:")
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

이 연습에서는 *데이터 저장소* 형식으로 데이터를 사용하기 위한 몇 가지 옵션을 살펴보았습니다.

Azure Machine Learning에서는 *데이터 세트* 형식의 데이터 추상화 수준이 추가로 제공됩니다. 데이터 세트에 대해서는 다음 연습에서 살펴볼 것입니다.